In [1]:
from coffea import hist
import math

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import mplhep as hep
plt.style.use(hep.style.CMS)

import awkward as ak

from matplotlib import colors
POPTS={'norm':colors.LogNorm()}

In [2]:
from data import getData, repackage
ldmx_dict = getData(chunks=True, fnames="/Users/chloeg/Desktop/Work/Fermilab2021/HistData/kaon_pn_4GeV_Jun24_ntuple/*.root")

In [3]:
def flat(x,axis=None): # for now must cast while waiting for coffea to catch up
        try:
            return ak.to_numpy(ak.flatten(x,axis=axis)) 
        except:
            return x

In [4]:
hists = {}
hists["lambda_hist"] = hist.Hist("Sim Lambda",
                                hist.Bin("e", r"Kinetic Energy [MeV]", 80, 0, 1000)
                            )
hists["lambda_kplus_hist"] = hist.Hist("Sim Lambda",
                                hist.Bin("e", r"Kinetic Energy [MeV]", 80, 0, 1000)
                            )

hists["lambda_kshort_hist"] = hist.Hist("Sim Lambda",
                                hist.Bin("e", r"Kinetic Energy [MeV]", 80, 0, 1000)
                            )
hists["lambda_klong_hist"] = hist.Hist("Sim Lambda",
                                hist.Bin("e", r"Kinetic Energy [MeV]", 80, 0, 1000)
                            )

#hists["lambda_electon_hist"] = hist.Hist("Sim Lambda",      
#                                hist.Bin("l", r"$\Lambda$ Kinetic Energy [MeV]", 40, 0, 3000),
#                                hist.Bin("e", r"Electron Energy [MeV]", 40, 0, 1500),
#                            )
#hists["lambda_electon_hist_n1"] = hist.Hist("Sim Lambda",      
#                                hist.Bin("l", r"$\Lambda$ Kinetic Energy [MeV]", 40, 0, 3000),
#                                hist.Bin("e", r"Electron Energy [MeV]", 40, 0, 1500),
#                            )


In [5]:
def ProcessChunk(chunk, hists):
    ldmx_events = repackage(chunk)

    sim_kaon = ldmx_events['Sim_Kaon']
    sim_kaon_n = ldmx_events['n']['Sim_Kaon']
    sim_lambda = ldmx_events['Sim_Lambda']
    sim_lambda_dau1 = ldmx_events['Sim_Lambda_dau1']
    sim_lambda_dau2 = ldmx_events['Sim_Lambda_dau2']
    
    lambda0_mass = 1115.683
    kaon_mass = 497.648
    #all lambda are 0 because they are the only light enough
    
    kaone_mask = sim_kaon.e >= 1440
    
    n1_mask = (abs(flat(sim_kaon_n)) == 1)
    kplus_mask = ((flat(sim_kaon[n1_mask].pdgID)) == 321)
    kshort_mask = ((flat(sim_kaon[n1_mask].pdgID)) == 310)
    klong_mask = ((flat(sim_kaon[n1_mask].pdgID)) == 130)
    
    lambda_mask = ((flat(sim_lambda[n1_mask][kplus_mask].pdgID)) == 3122)
    
    lambda_energy_plus = (flat(sim_lambda[n1_mask][kplus_mask].e))
    kaon_energy_plus = (flat(sim_kaon[n1_mask][kplus_mask][lambda_mask].e))
    electron_energy_plus = (flat(sim_kaon[n1_mask][kplus_mask][lambda_mask].electrone))
    
    system_energy_plus = lambda_energy_plus + kaon_energy_plus + electron_energy_plus
    
    system_energy_mask_plus = (flat(system_energy_plus) <= 4000)
    print(system_energy_plus.shape)
    print(flat(system_energy_plus[system_energy_mask_plus]).shape)
    
    plus_acceptance = flat(system_energy_plus[system_energy_mask_plus].shape/system_energy_plus.shape
    
    hists["lambda_hist"].fill(
                         e=flat(sim_lambda.e)-lambda0_mass,
                         )   
    hists["lambda_kplus_hist"].fill(
                         e=flat(sim_lambda[n1_mask][kplus_mask].e)-lambda0_mass,
                         ) 
    hists["lambda_kshort_hist"].fill(
                         e=flat(sim_lambda[n1_mask][kshort_mask].e)-lambda0_mass,
                         ) 
    hists["lambda_klong_hist"].fill(
                         e=flat(sim_lambda[n1_mask][klong_mask].e)-lambda0_mass,
                         ) 
    
    


In [6]:
nchunk = 0
for chunk in ldmx_dict:
    nchunk += 1
    print('process',nchunk)
    ProcessChunk(chunk, hists)


process 1
(56,)
(54,)


TypeError: unsupported operand type(s) for /: 'tuple' and 'tuple'

In [ ]:
print(hists["lambda_hist"])
fig, ax = plt.subplots()
hist.plot1d(hists["lambda_hist"],ax=ax,clear=False);
leg = ax.legend([r'$\Lambda$'])

print(hists["lambda_hist"])
fig, ax = plt.subplots()
hist.plot1d(hists["lambda_kplus_hist"],ax=ax,clear=False);
leg = ax.legend([r'$\Lambda$ produced with 1 $K^+$'])

print(hists["lambda_hist"])
fig, ax = plt.subplots()
hist.plot1d(hists["lambda_kshort_hist"],ax=ax,clear=False);
leg = ax.legend([r'$\Lambda$ produced with 1 $K_s$'])


print(hists["lambda_hist"])
fig, ax = plt.subplots()
hist.plot1d(hists["lambda_klong_hist"],ax=ax,clear=False);
leg = ax.legend([r'$\Lambda$ produced with 1 $K_L$'])


In [ ]:
#print(hists["lambda_electon_hist"])

fig, ax = plt.subplots(1,3,figsize=(24,6), constrained_layout=True)
hist.plot2d(hists["lambda_electon_hist"], xaxis='l', ax=ax[0],patch_opts=POPTS);
hist.plot2d(hists["lambda_electon_hist_n1"], xaxis='l', ax=ax[1],patch_opts=POPTS);
hist.plot2d(hists["lambda_electon_hist_n2"], xaxis='l', ax=ax[2],patch_opts=POPTS);
ax[0].text(2500, 50,r'All n')
ax[1].text(2500, 50,r'n=1')
ax[2].text(2500, 50,r'n=2')
